<a href="https://colab.research.google.com/github/parth-mango/YOLOv3_scratch/blob/main/yolov3_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/

/content


In [2]:
# !pip install PyYAML==5.4.1
# !pip install -q wandb

     |████████████████████████████████| 636 kB 6.8 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.7 MB 8.2 MB/s 
     |████████████████████████████████| 97 kB 7.1 MB/s 
     |████████████████████████████████| 140 kB 52.1 MB/s 
     |████████████████████████████████| 180 kB 50.1 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 


In [3]:
import logging
import yaml
from pathlib import Path
import glob
import re
import os
import wandb

In [21]:
LOCAL_RANK= int(os.getenv('LOCAL_RANK', -1)) #os.getenv() method in Python returns the value of the environment variable key if it exists otherwise returns the default value.
RANK = int(os.getenv('RANK', -1)) 
WORLD_SIZE = int(os.getenv('WORLD_SIZE', 1))

In [4]:
# FILE= Path(__file__).resolve()
ROOT= Path('/content/')

In [5]:
!mkdir utils
%cd /content/utils/
!mkdir loggers
%cd /content/utils/loggers/
!mkdir wandb
%cd /content/
!mkdir data
%cd /content/data/
!mkdir scripts
%cd /content/
!mkdir datasets
%cd /content/

/content/utils
/content/utils/loggers
/content
/content/data
/content
/content


In [6]:
# # Upload folllowing files: 
# # 1. init.py for loggers
# # 2. general.py 
# !mv general.py /content/utils
# !mv __init__.py /content/utils/loggers
# !mv wandb_utils.py /content/utils/loggers/wandb/
# !mv coco128.yaml /content/data/
# !mv get_coco128.sh /content/data/scripts/
!rm -rf /content/runs

In [7]:
!bash /content/data/scripts/get_coco128.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   650  100   650    0     0   2425      0 --:--:-- --:--:-- --:--:--  2416
100 6820k  100 6820k    0     0  11.3M      0 --:--:-- --:--:-- --:--:-- 11.3M


In [8]:
from utils.general import colorstr
from utils.loggers import Loggers
from utils.general import LOGGER, methods

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [9]:
import argparse

In [10]:
parser = argparse.ArgumentParser()
parser.add_argument('--weights', type=str, default=ROOT / 'yolov3.pt', help='initial weights path')
parser.add_argument('--cfg', type=str, default='', help='model.yaml path')
parser.add_argument('--data', type=str, default=ROOT / 'data/coco128.yaml', help='dataset.yaml path')
parser.add_argument('--hyp', type=str, default=ROOT / 'data/hyps/hyp.scratch.yaml', help='hyperparameters path')
parser.add_argument('--epochs', type=int, default=300)
parser.add_argument('--batch-size', type=int, default=16, help='total batch size for all GPUs, -1 for autobatch')
parser.add_argument('--imgsz', '--img', '--img-size', type=int, default=640, help='train, val image size (pixels)')
parser.add_argument('--rect', action='store_true', help='rectangular training')
parser.add_argument('--resume', nargs='?', const=True, default=False, help='resume most recent training')
parser.add_argument('--nosave', action='store_true', help='only save final checkpoint')
parser.add_argument('--noval', action='store_true', help='only validate final epoch')
parser.add_argument('--noautoanchor', action='store_true', help='disable autoanchor check')
parser.add_argument('--evolve', type=int, nargs='?', const=300, help='evolve hyperparameters for x generations')
parser.add_argument('--bucket', type=str, default='', help='gsutil bucket')
parser.add_argument('--cache', type=str, nargs='?', const='ram', help='--cache images in "ram" (default) or "disk"')
parser.add_argument('--image-weights', action='store_true', help='use weighted image selection for training')
parser.add_argument('--device', default='', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
parser.add_argument('--multi-scale', action='store_true', help='vary img-size +/- 50%%')
parser.add_argument('--single-cls', action='store_true', help='train multi-class data as single-class')
parser.add_argument('--adam', action='store_true', help='use torch.optim.Adam() optimizer')
parser.add_argument('--sync-bn', action='store_true', help='use SyncBatchNorm, only available in DDP mode')
parser.add_argument('--workers', type=int, default=8, help='max dataloader workers (per RANK in DDP mode)')
parser.add_argument('--project', default=ROOT / 'runs/train', help='save to project/name')
parser.add_argument('--name', default='exp', help='save to project/name')
parser.add_argument('--exist-ok', action='store_true', help='existing project/name ok, do not increment')
parser.add_argument('--quad', action='store_true', help='quad dataloader')
parser.add_argument('--linear-lr', action='store_true', help='linear LR')
parser.add_argument('--label-smoothing', type=float, default=0.0, help='Label smoothing epsilon')
parser.add_argument('--patience', type=int, default=100, help='EarlyStopping patience (epochs without improvement)')
parser.add_argument('--freeze', type=int, default=0, help='Number of layers to freeze. backbone=10, all=24')
parser.add_argument('--save-period', type=int, default=-1, help='Save checkpoint every x epochs (disabled if < 1)')
parser.add_argument('--local_rank', type=int, default=-1, help='DDP parameter, do not modify')

# Weights & Biases arguments
parser.add_argument('--entity', default=None, help='W&B: Entity')
parser.add_argument('--upload_dataset', action='store_true', help='W&B: Upload dataset as artifact table')
parser.add_argument('--bbox_interval', type=int, default=-1, help='W&B: Set bounding-box image logging interval')
parser.add_argument('--artifact_alias', type=str, default='latest', help='W&B: Version of dataset artifact to use')
opt = parser.parse_args("")

In [11]:
def increment_path(path, exist_ok= False, sep= '', mkdir= False):
  #Increment file or directory path i.e. runs/exp --> runs/exp{sep}2 --> runs/exp{sep}3, ...etc
  path = Path(path)
  if path.exists() and not exist_ok:
    
    path, suffix= (path.with_suffix(''), path.suffix) if path.is_file() else (path, '')
    dirs= glob.glob(f"{path}{sep}*")
    matches= [re.search(rf"%s{sep}(\d+)" % path.stem, d) for d in dirs]
    i = [int(m.groups()[0]) for m in matches if m] #indices
    n= max(i) + 1 if i else 2 # increment number
    path= Path(f"{path}{sep}{n}{suffix}") #increment path
  if mkdir:
    path.mkdir(parents= True, exist_ok= True) #make directory
  return path 
    

In [12]:
increment_path(Path(opt.project)) 

PosixPath('/content/runs/train')

In [13]:
opt.save_dir = str(increment_path(Path(opt.project) / opt.name, exist_ok=opt.exist_ok ))

In [14]:
save_dir, epochs, batch_size, weights, single_cls, evolve, data, cfg, resume, noval, nosave, workers, freeze = \
Path(opt.save_dir), opt.epochs, opt.batch_size, opt.weights, \
opt.single_cls, opt.evolve, opt.data, opt.cfg, opt.resume, opt.noval, opt.nosave, opt.workers, opt.freeze    

In [15]:
w= save_dir / 'weights'  #weights dir
(w.parent if evolve else w ).mkdir(parents= True, exist_ok= True)
last, best = w / 'last.pt', w / 'best.pt'


In [16]:
hyp_f= '/content/hyp.scratch.yaml'

In [17]:

with open(hyp_f) as f:
        hyp = yaml.safe_load(f)

In [18]:
LOGGER.info(colorstr('hyperparameters:') + ', '.join(f'{k}={v}' for k, v in hyp.items() ))

hyperparameters:lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, mosaic=1.0, mixup=0.0, copy_paste=0.0


In [19]:
with open(save_dir / 'hyp.yaml' , 'w') as f:
  yaml.safe_dump(hyp, f, sort_keys= False)
with open(save_dir / 'opt.yaml', 'w') as f:
  yaml.dump(vars(opt), f, sort_keys= False)   #conf1

data_dict= None  

In [20]:
#Loggers
if RANK in [-1, 0]:
  loggers= Loggers(save_dir, weights, opt, hyp, logger= LOGGER)  
  if loggers.wandb:
    data_dict= loggers.wandb.data_dict
    if resume:
      weights, epochs, hyp= opt.weights, opt.epochs, opt.hyp

  #Register actions
  for k in methods(loggers):
    callbacks.register_action(k, callback= getattr(loggers, k))

NameError: ignored